In [107]:
import sys
sys.path.append("../../utils/")

import requests
from bs4 import BeautifulSoup

from decode_url import parse_url

In [108]:
class WikiIntroScrapper:
    
    def __init__(self, url):
        self.url = url
        self.title = None
        self.intro_links = []
        self.parsed_links = []
    
    def _get_article(self):
        resp = requests.get(self.url)
        soup = BeautifulSoup(resp.content)
        self._set_title(soup)
        return soup
        
    def _set_title(self, soup):
        self.title = soup.find_all("h1")[0].text

    
    def _get_intro_links(self):
        soup = self._get_article().find_all(class_="mw-parser-output")[0]
        
        children = list(soup.children)
        
        started_intro_text = False
        
        for child in children:
            if child.name == "p" and child.has_attr("class") == False:
                started_intro_text = True
                self.intro_links += child.find_all("a")
            if child.name != "p" and started_intro_text:
                break
                
    def parse_intro_links(self):
        self._get_intro_links()
        
        for link in self.intro_links:
            current_href = link.get('href')
            if current_href.startswith("/wiki/") and not ("Help:" in current_href):
                self.parsed_links.append(parse_url("https://en.wikipedia.org" + current_href))
                
        return self.parsed_links

In [109]:
ws = WikiIntroScrapper("https://en.wikipedia.org/wiki/Für_Elise")

ws.parse_intro_links()

print("Title:\t", ws.title)

for link in ws.parsed_links:
    print(link)

Title:	 Für Elise
https://en.wikipedia.org/wiki/A_minor
https://en.wikipedia.org/wiki/WoO_(Beethoven)
https://en.wikipedia.org/wiki/Biamonti_Catalogue
https://en.wikipedia.org/wiki/Piano
https://en.wikipedia.org/wiki/Ludwig_van_Beethoven
https://en.wikipedia.org/wiki/Ludwig_Nohl
https://en.wikipedia.org/wiki/Bagatelle_(music)
https://en.wikipedia.org/wiki/Album_leaf
https://en.wikipedia.org/wiki/Therese_Malfatti
https://en.wikipedia.org/wiki/Elisabeth_Röckel
https://en.wikipedia.org/wiki/Elise_Barensfeld
